In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import os
from run_nano.sample_list import sample_df
from run_nano.crab_cfg_templates import mc_template, data_template
from run_nano.job_template import job_mc_template, job_data_template, job_mc_local_template

In [ ]:
template_meta = {
    2016:
    {   "era": 2016, 
        "triggers": ['HLT_Mu50','HLT_TkMu50', 'HLT_DoubleEle33_CaloIdL_MW', 'HLT_DoubleEle33_CaloIdL_GsfTrkIdVL']  ,
        "json": "Cert_271036-284044_13TeV_ReReco_07Aug2017_Collisions16_JSON.txt"
    },
    2017:
    {   "era": 2017, 
        "triggers": ['HLT_Mu50','HLT_OldMu100','HLT_TkMu100', 'HLT_DoubleEle33_CaloIdL_MW', 'HLT_DoubleEle25_CaloIdL_MW'] ,
        "json": "Cert_294927-306462_13TeV_EOY2017ReReco_Collisions17_JSON.txt"
    },
    2018:
    {   "era": 2018, 
        "triggers": ['HLT_Mu50','HLT_OldMu100','HLT_TkMu100', 'HLT_DoubleEle25_CaloIdL_MW']  ,
        "json": "Cert_314472-325175_13TeV_17SeptEarlyReReco2018ABC_PromptEraD_Collisions18_JSON.txt"
    }
}

In [ ]:
#set up directories for each era
def write_file(template, meta_dict, filename):
    '''This function makes jopy python files for each era, customizing the triggers and eras.'''
    with open (filename, 'w') as f:
        f.write(template.format(era=meta_dict['era'], triggers=meta_dict['triggers']))

for era in [2016, 2017, 2018]:
    #make version of the job python file in each era directory
    #you might need to make 2016-2018 dirs
    write_file(job_mc_template, template_meta[era], "{}/job_mc_crab.py".format(era))
    write_file(job_data_template, template_meta[era], "{}/job_data_crab.py".format(era))
    write_file(job_mc_local_template, template_meta[era], "{}/job_mc_local.py".format(era))
    #copy other needed files. Tried soft link, but it had some weird permission issues.
    for f in ['crab_script_data.sh', 'crab_script_mc.sh', 'keep_and_drop_bff.txt', 'PSet.py']:
        os.system('rm {}/{}'.format(era, f))
        os.system('cp {f} {e}/{f}'.format(f=f, e=era))

In [ ]:
#make custom configs for each dataset
for i, row in sample_df.iterrows():
    # set up data needed to fill template
    isMC = row['mc']
    era = row['era']
    json = template_meta[era]['json']
    das = row['das']
    _, samplename, metaname, datatier = das.split('/')
    requestName = "{}_{}_{}_{}".format(era, row['type'], row['name'], samplename)
    outputName = row['name']
    # select the right template
    if isMC: crab_template = mc_template
    else: crab_template = data_template
    #formate template
    formated_template = crab_template.format(
        requestName=requestName,
        outputName=outputName,
        era=era,
        das=das,
        json=json
    )
    #write file
    with open ('{}/{}.py'.format(era, outputName), 'w') as f:
        f.write(formated_template)